In [1]:
#导入相关包
import scipy
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import datetime
import time

#### 数据读入，并将初赛决赛相同数据合并concat

In [2]:
# 历史入库数据
ruku_data = pd.read_excel('./初赛/入库流量数据.xlsx')
ruku_data_jue = pd.read_excel('./决赛/入库流量数据.xlsx')
# 数据合并
ruku_data = pd.concat([ruku_data,ruku_data_jue],axis=0)
#时间类型转换
ruku_data['TimeStample_cov'] = pd.to_datetime(ruku_data['TimeStample'])

# 环境数据，温度，风速，同上
huanjing_data = pd.read_excel('./初赛/环境表.xlsx')
huanjing_data_jue = pd.read_excel('./决赛/环境表.xlsx')
huanjing_data = pd.concat([huanjing_data,huanjing_data_jue],axis=0)
huanjing_data.columns = ['TimeStample','气温','风速','风向']
huanjing_data['daysd'] = pd.to_datetime(huanjing_data['TimeStample'])

# 未来5天天气预报,降水量
tianqiyubao_data = pd.read_excel('./初赛/降雨预报数据.xlsx')
tianqiyubao_data_jue = pd.read_excel('./决赛/降雨预报数据.xlsx')
tianqiyubao_data = pd.concat([tianqiyubao_data,tianqiyubao_data_jue],axis=0)
tianqiyubao_data['TimeStample_cov'] = pd.to_datetime(tianqiyubao_data['TimeStample'])


#遥测站数据
yaocezhan_data = pd.read_excel('./初赛/遥测站降雨数据.xlsx')
yaocezhan_data_jue = pd.read_excel('./决赛/遥测站降雨数据.xlsx')
yaocezhan_data = pd.concat([yaocezhan_data,yaocezhan_data_jue],axis=0)
yaocezhan_data['TimeStample_cov'] = pd.to_datetime(yaocezhan_data['TimeStample'])
# 求得当天遥测站的sum
yaocezhan_data['all_rain'] = yaocezhan_data[['R'+str(i+1) for i in range(39)]].sum(axis=1)
yaocezhan_data['mean_rain'] = yaocezhan_data[['R'+str(i+1) for i in range(39)]].sum(axis=1)
yaocezhan_data.set_index('TimeStample_cov', inplace=True)
# 将遥测站数据转换成3小时的时间段，方便数据进行关联
yaocezhan_data_resample = yaocezhan_data.resample('3H').sum().reset_index()
yaocezhan_data_resample['TimeStample_cov'] = yaocezhan_data_resample['TimeStample_cov']+datetime.timedelta(hours=2)

### 特征工程，测试集、训练集构造

In [3]:
# 决赛预测时段
# 1.【2019年2月1日至2月7日】
# 2.【2019年4月1日至4月7日】
# 3.【2019年6月1日至6月7日】
# 4.【2019年8月1日至8月7日】
# 5.【2019年11月1日至11月7日】

In [4]:
# 训练集构造
create_df = ruku_data.copy()
create_df = create_df.merge(yaocezhan_data_resample[['TimeStample_cov','all_rain','mean_rain']],how='left',on='TimeStample_cov')
# create_df.Qi.fillna(0,inplace=True)

#创建分段一测试数据集
cut_1 = pd.DataFrame()
cut_1['TimeStample_cov'] = pd.date_range('2019-02-01 02:00:00', '2019-02-08', freq='3h')
cut_1 = cut_1.merge(yaocezhan_data_resample[['TimeStample_cov','all_rain','mean_rain']],how='left',on='TimeStample_cov')

#创建分段二测试数据集
cut_2 = pd.DataFrame()
cut_2['TimeStample_cov'] = pd.date_range('2019-04-01 02:00:00', '2019-04-08', freq='3h')
cut_2 = cut_2.merge(yaocezhan_data_resample[['TimeStample_cov','all_rain','mean_rain']],how='left',on='TimeStample_cov')

#创建分段三测试数据集
cut_3 = pd.DataFrame()
cut_3['TimeStample_cov'] = pd.date_range('2019-06-01 02:00:00', '2019-06-08', freq='3h')
cut_3 = cut_3.merge(yaocezhan_data_resample[['TimeStample_cov','all_rain','mean_rain']],how='left',on='TimeStample_cov')
cut_3 = cut_3.fillna(0)

#创建分段四测试数据集
cut_4 = pd.DataFrame()
cut_4['TimeStample_cov'] = pd.date_range('2019-08-01 02:00:00', '2019-08-08', freq='3h')
cut_4 = cut_4.merge(yaocezhan_data_resample[['TimeStample_cov','all_rain','mean_rain']],how='left',on='TimeStample_cov')
cut_4 = cut_4.fillna(0)

#创建分段五 测试数据集
cut_5 = pd.DataFrame()
cut_5['TimeStample_cov'] = pd.date_range('2019-11-01 02:00:00', '2019-11-08', freq='3h')
cut_5 = cut_5.merge(yaocezhan_data_resample[['TimeStample_cov','all_rain','mean_rain']],how='left',on='TimeStample_cov')
cut_5 = cut_5.fillna(0)

### 数据时间转换

In [5]:
# 转换时间
create_df['daysd'] = pd.to_datetime(create_df.TimeStample_cov.map(lambda x:x.strftime('%Y-%m-%d')))
cut_2['daysd'] = pd.to_datetime(cut_2.TimeStample_cov.map(lambda x:x.strftime('%Y-%m-%d')))
cut_1['daysd'] = pd.to_datetime(cut_1.TimeStample_cov.map(lambda x:x.strftime('%Y-%m-%d')))
cut_3['daysd'] = pd.to_datetime(cut_3.TimeStample_cov.map(lambda x:x.strftime('%Y-%m-%d')))
cut_4['daysd'] = pd.to_datetime(cut_4.TimeStample_cov.map(lambda x:x.strftime('%Y-%m-%d')))
cut_5['daysd'] = pd.to_datetime(cut_5.TimeStample_cov.map(lambda x:x.strftime('%Y-%m-%d')))

In [6]:
# 前3天天气预报进行关联，只是用前3天。
tianqiyubao_data = pd.read_excel('./初赛/降雨预报数据.xlsx')
tianqiyubao_data['TimeStample'] = pd.to_datetime(tianqiyubao_data['TimeStample'])
tianqiyubao_data.columns = ['daysd','D1','D2','D3','D4','D5']
tianqiyubao_data['daysd'] = tianqiyubao_data['daysd']+datetime.timedelta(days=3)
create_df = create_df.merge(tianqiyubao_data[['daysd','D1','D2','D3']],how='left',on='daysd')

cut_1 = cut_1.merge(tianqiyubao_data[['daysd','D1','D2','D3']],how='left',on='daysd')
cut_2 = cut_2.merge(tianqiyubao_data[['daysd','D1','D2','D3']],how='left',on='daysd')
cut_3 = cut_3.merge(tianqiyubao_data[['daysd','D1','D2','D3']],how='left',on='daysd')
cut_4 = cut_4.merge(tianqiyubao_data[['daysd','D1','D2','D3']],how='left',on='daysd')
cut_5 = cut_5.merge(tianqiyubao_data[['daysd','D1','D2','D3']],how='left',on='daysd')

### 关联环境因素

In [7]:
#测试机训练集，关联环境因素
create_df = create_df.merge(huanjing_data[['daysd','气温','风速']],how='left',on='daysd')
cut_1 = cut_1.merge(huanjing_data[['daysd','气温','风速']],how='left',on='daysd')

cut_2 = cut_2.merge(huanjing_data[['daysd','气温','风速']],how='left',on='daysd')

cut_3 = cut_3.merge(huanjing_data[['daysd','气温','风速']],how='left',on='daysd')

cut_4 = cut_4.merge(huanjing_data[['daysd','气温','风速']],how='left',on='daysd')

cut_5 = cut_5.merge(huanjing_data[['daysd','气温','风速']],how='left',on='daysd')

### 关联前8个时间段数据

In [8]:
create_df['Qi_1'] = create_df.Qi.shift(1)
create_df['Qi_2'] = create_df.Qi.shift(2)
create_df['Qi_3'] = create_df.Qi.shift(3)
create_df['Qi_4'] = create_df.Qi.shift(4)
create_df['Qi_5'] = create_df.Qi.shift(5)
create_df['Qi_6'] = create_df.Qi.shift(6)
create_df['Qi_7'] = create_df.Qi.shift(7)
create_df['Qi_8'] = create_df.Qi.shift(8)

### 手动填写各个预测分段前面8个时间点的数据，比如2月1号，就填写1月31号的后8个时间点数据

In [9]:
# 可以使用该语句进行查看前8个时间点
# create_df[create_df.TimeStample>='2019-10-31'][:8].Qi
# 填写时间段1
cut_1['Qi_1'] = 0.081589
cut_1['Qi_2'] = 0.049254
cut_1['Qi_3'] = 0.021640
cut_1['Qi_4'] = 0.005774
cut_1['Qi_5'] = 0.191796
cut_1['Qi_6'] = 0.171462
cut_1['Qi_7'] = 0.133404
cut_1['Qi_8'] = 0.141035
# 时间段2
cut_2['Qi_1'] = 0.238289
cut_2['Qi_2'] = 0.153286
cut_2['Qi_3'] = 0.103078
cut_2['Qi_4'] = 0.088517
cut_2['Qi_5'] = 0.244264
cut_2['Qi_6'] = 0.289050
cut_2['Qi_7'] = 0.356580
cut_2['Qi_8'] = 0.346287

cut_3['Qi_1'] = 0.179043
cut_3['Qi_2'] = 0.027514
cut_3['Qi_3'] = 0.043480
cut_3['Qi_4'] = 0.051162
cut_3['Qi_5'] = 0.050158
cut_3['Qi_6'] = 0.118391
cut_3['Qi_7'] = 0.107245
cut_3['Qi_8'] = 0.045338

cut_4['Qi_1'] = 0.042526
cut_4['Qi_2'] = 0.014611
cut_4['Qi_3'] = 0.014611
cut_4['Qi_4'] = 0.015012
cut_4['Qi_5'] = 0.033188
cut_4['Qi_6'] = 0.028518
cut_4['Qi_7'] = 0.028518
cut_4['Qi_8'] = 0.075011

cut_5['Qi_1'] = 0.188281
cut_5['Qi_2'] = 0.411458
cut_5['Qi_3'] = 0.293920
cut_5['Qi_4'] = 0.079681
cut_5['Qi_5'] = 0.214942
cut_5['Qi_6'] = 0.077723
cut_5['Qi_7'] = 0.294472
cut_5['Qi_8'] = 0.193403

### 年月日特征、以及数据排序特征，保留数据顺序

In [10]:
# 年、月、日、小时排序特征
create_df['year'] = create_df.TimeStample_cov.dt.year
create_df['month'] = create_df.TimeStample_cov.dt.month
create_df['hour'] = create_df.TimeStample_cov.dt.hour
create_df['hourIndex'] = create_df.apply(lambda x:time.mktime(x['TimeStample_cov'].timetuple()),axis=1 )
create_df['hourIndex'] = create_df.apply(lambda x: int(x['hourIndex'] /(3600)) ,axis=1 ) 

#各个分段同上
cut_1['year'] = cut_1.TimeStample_cov.dt.year
cut_1['month'] = cut_1.TimeStample_cov.dt.month
cut_1['hour'] = cut_1.TimeStample_cov.dt.hour
cut_1['hourIndex'] = cut_1.apply(lambda x:time.mktime(x['TimeStample_cov'].timetuple()),axis=1 )
cut_1['hourIndex'] = cut_1.apply(lambda x: int(x['hourIndex'] /(3600)) ,axis=1 ) 

cut_2['year'] = cut_2.TimeStample_cov.dt.year
cut_2['month'] = cut_2.TimeStample_cov.dt.month
cut_2['hour'] = cut_2.TimeStample_cov.dt.hour
cut_2['hourIndex'] = cut_2.apply(lambda x:time.mktime(x['TimeStample_cov'].timetuple()),axis=1 )
cut_2['hourIndex'] = cut_2.apply(lambda x: int(x['hourIndex'] /(3600)) ,axis=1 ) 

cut_3['year'] = cut_3.TimeStample_cov.dt.year
cut_3['month'] = cut_3.TimeStample_cov.dt.month
cut_3['hour'] = cut_3.TimeStample_cov.dt.hour
cut_3['hourIndex'] = cut_3.apply(lambda x:time.mktime(x['TimeStample_cov'].timetuple()),axis=1 )
cut_3['hourIndex'] = cut_3.apply(lambda x: int(x['hourIndex'] /(3600)) ,axis=1 )

cut_4['year'] = cut_4.TimeStample_cov.dt.year
cut_4['month'] = cut_4.TimeStample_cov.dt.month
cut_4['hour'] = cut_4.TimeStample_cov.dt.hour
cut_4['hourIndex'] = cut_4.apply(lambda x:time.mktime(x['TimeStample_cov'].timetuple()),axis=1 )
cut_4['hourIndex'] = cut_4.apply(lambda x: int(x['hourIndex'] /(3600)) ,axis=1 )

cut_5['year'] = cut_5.TimeStample_cov.dt.year
cut_5['month'] = cut_5.TimeStample_cov.dt.month
cut_5['hour'] = cut_5.TimeStample_cov.dt.hour
cut_5['hourIndex'] = cut_5.apply(lambda x:time.mktime(x['TimeStample_cov'].timetuple()),axis=1 )
cut_5['hourIndex'] = cut_5.apply(lambda x: int(x['hourIndex'] /(3600)) ,axis=1 )

In [11]:
# 训练集使用的特征筛选
usecol = [i for i in create_df.columns if i not in['TimeStample','TimeStample_cov','daysd']]
end_usecol = [i for i in create_df.columns if i not in['TimeStample','TimeStample_cov','daysd','Qi']]

In [12]:
# 需要使用的数据
usedata = create_df[usecol]
usedata = usedata.dropna()
usedata.reset_index(drop=True,inplace=True)

### 构建一个lightgbm方法模型，采用五折交叉验证

In [13]:

def lgb_model(train_data,y,N):
    # 超参数
    lgb_param_l1 = { 'learning_rate': 0.01, 
                    'boosting_type': 'gbdt', 
                    'objective': 'regression_l1',
                    'metric': 'mae',
                    'min_child_samples': 46, 
                    'min_child_weight': 0.01,
                    'feature_fraction': 0.8, 
                    'bagging_fraction': 0.8, 
                    'bagging_freq': 2, 
                    'num_leaves': 16, 
                    'max_depth': 5, 
                    'n_jobs': -1, 
                    'seed': 2019, }
    # 交叉验证
    skf=KFold(n_splits=N,shuffle=True,random_state=42)
    oof_lgb=np.zeros(train_data.shape[0])
    models = []
    for i,(tr,va) in enumerate(skf.split(train_data,y)):
        print('fold:',i+1,'training')
        X_train, X_test, y_train, y_test = train_data[tr],train_data[va],y[tr],y[va]
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_valid = lgb.Dataset(X_test, y_test, reference=lgb_train)
        clf = lgb.train(lgb_param_l1,lgb_train, 
                         valid_sets=[lgb_valid],#, xgtrain], 
                         valid_names=['valid'],#,'train'], 
                         num_boost_round=30000,
                         early_stopping_rounds=100,
                         verbose_eval=50)
        oof_lgb[va] += clf.predict(X_test,num_iteration=clf.best_iteration)
        models.append(clf)
    print("stacking的score: {:<8.8f}".format(np.sqrt(mean_squared_error(oof_lgb, y))))
    return oof_lgb,models

In [14]:
# 使用全量数据进行模型训练
oof_lgb,models=lgb_model(usedata[end_usecol].values,usedata.Qi,5)

fold: 1 training
Training until validation scores don't improve for 100 rounds
[50]	valid's l1: 0.108438
[100]	valid's l1: 0.0824995
[150]	valid's l1: 0.068932
[200]	valid's l1: 0.0617115
[250]	valid's l1: 0.0577688
[300]	valid's l1: 0.0552005
[350]	valid's l1: 0.0540969
[400]	valid's l1: 0.0534904
[450]	valid's l1: 0.0530915
[500]	valid's l1: 0.0528377
[550]	valid's l1: 0.0526683
[600]	valid's l1: 0.0525351
[650]	valid's l1: 0.0524665
[700]	valid's l1: 0.052414
[750]	valid's l1: 0.0523598
[800]	valid's l1: 0.0523144
[850]	valid's l1: 0.0522837
[900]	valid's l1: 0.0522497
[950]	valid's l1: 0.052223
[1000]	valid's l1: 0.0521967
[1050]	valid's l1: 0.0521812
[1100]	valid's l1: 0.052163
[1150]	valid's l1: 0.0521579
[1200]	valid's l1: 0.0521395
[1250]	valid's l1: 0.0521228
[1300]	valid's l1: 0.0521214
[1350]	valid's l1: 0.0521075
[1400]	valid's l1: 0.0520992
[1450]	valid's l1: 0.0520903
[1500]	valid's l1: 0.0520873
[1550]	valid's l1: 0.052086
[1600]	valid's l1: 0.0520741
[1650]	valid's l1: 

In [48]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

from matplotlib.font_manager import FontProperties
myfont=FontProperties(fname=r'C:\Windows\Fonts\simhei.ttf',size=14)
sns.set(font=myfont.get_name())
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [51]:

# feature_imp = pd.DataFrame({'Value':models[0].feature_importance(),'Feature':end_usecol})
# plt.figure(figsize=(40, 20))
# sns.set(font_scale = 5)
# sns.set_style('whitegrid', {'font.sans-serif': ['simhei', 'Arial']})
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[0:20])
# plt.title('LightGBM Features (avg over folds)')
# plt.tight_layout()
# plt.savefig('lgbm_importances-01.png')
# plt.show()


### 分段一 预测

In [15]:
# 结果存放到res中，分别对56个值进行预测，上一个时间段的预测，作为下一个时间段的输入，更新前8个时间点特征
res = []
for i in range(56):
    tv = 0
    #五折交叉后的模型取平均
    for model in models:
        pred = model.predict(cut_1[i:1+i][end_usecol].values)[0]
        tv+=pred
    ev = tv/len(models)
    res.append(ev)
    cut_1.loc[i+1,'Qi_1'] = ev
    cut_1.loc[i+1,'Qi_2'] = cut_1.loc[i,'Qi_1']
    cut_1.loc[i+1,'Qi_3'] = cut_1.loc[i,'Qi_2']
    cut_1.loc[i+1,'Qi_4'] = cut_1.loc[i,'Qi_3']
    cut_1.loc[i+1,'Qi_5'] = cut_1.loc[i,'Qi_4']
    cut_1.loc[i+1,'Qi_6'] = cut_1.loc[i,'Qi_5']
    cut_1.loc[i+1,'Qi_7'] = cut_1.loc[i,'Qi_6']
    cut_1.loc[i+1,'Qi_8'] = cut_1.loc[i,'Qi_7']

### 分段2预测，注释思路同分段一

In [16]:
res2 = []
for i in range(56):
    tv = 0
    for model in models:
        pred = model.predict(cut_2[i:1+i][end_usecol].values)[0]
        tv+=pred
    ev = tv/len(models)
    res2.append(ev)
    cut_2.loc[i+1,'Qi_1'] = ev
    cut_2.loc[i+1,'Qi_2'] = cut_2.loc[i,'Qi_1']
    cut_2.loc[i+1,'Qi_3'] = cut_2.loc[i,'Qi_2']
    cut_2.loc[i+1,'Qi_4'] = cut_2.loc[i,'Qi_3']
    cut_2.loc[i+1,'Qi_5'] = cut_2.loc[i,'Qi_4']
    cut_2.loc[i+1,'Qi_6'] = cut_2.loc[i,'Qi_5']
    cut_2.loc[i+1,'Qi_7'] = cut_2.loc[i,'Qi_6']
    cut_2.loc[i+1,'Qi_8'] = cut_2.loc[i,'Qi_7']

### 分段3预测，注释思路同分段一

In [17]:
res3 = []
for i in range(56):
    tv = 0
    for model in models:
        pred = model.predict(cut_3[i:1+i][end_usecol].values)[0]
        tv+=pred
    ev = tv/len(models)
    res3.append(ev)
    cut_3.loc[i+1,'Qi_1'] = ev
    cut_3.loc[i+1,'Qi_2'] = cut_3.loc[i,'Qi_1']
    cut_3.loc[i+1,'Qi_3'] = cut_3.loc[i,'Qi_2']
    cut_3.loc[i+1,'Qi_4'] = cut_3.loc[i,'Qi_3']
    cut_3.loc[i+1,'Qi_5'] = cut_3.loc[i,'Qi_4']
    cut_3.loc[i+1,'Qi_6'] = cut_3.loc[i,'Qi_5']
    cut_3.loc[i+1,'Qi_7'] = cut_3.loc[i,'Qi_6']
    cut_3.loc[i+1,'Qi_8'] = cut_3.loc[i,'Qi_7']

### 分段4预测，注释思路同分段一

In [18]:
res4 = []
for i in range(56):
    tv = 0
    for model in models:
        pred = model.predict(cut_4[i:1+i][end_usecol].values)[0]
        tv+=pred
    ev = tv/len(models)
    res4.append(ev)
    cut_4.loc[i+1,'Qi_1'] = ev
    cut_4.loc[i+1,'Qi_2'] = cut_4.loc[i,'Qi_1']
    cut_4.loc[i+1,'Qi_3'] = cut_4.loc[i,'Qi_2']
    cut_4.loc[i+1,'Qi_4'] = cut_4.loc[i,'Qi_3']
    cut_4.loc[i+1,'Qi_5'] = cut_4.loc[i,'Qi_4']
    cut_4.loc[i+1,'Qi_6'] = cut_4.loc[i,'Qi_5']
    cut_4.loc[i+1,'Qi_7'] = cut_4.loc[i,'Qi_6']
    cut_4.loc[i+1,'Qi_8'] = cut_4.loc[i,'Qi_7']

### 分段5预测，注释思路同分段一

In [19]:
res5 = []
for i in range(56):
    tv = 0
    for model in models:
        pred = model.predict(cut_5[i:1+i][end_usecol].values)[0]
        tv+=pred
    ev = tv/len(models)
    res5.append(ev)
    cut_5.loc[i+1,'Qi_1'] = ev
    cut_5.loc[i+1,'Qi_2'] = cut_5.loc[i,'Qi_1']
    cut_5.loc[i+1,'Qi_3'] = cut_5.loc[i,'Qi_2']
    cut_5.loc[i+1,'Qi_4'] = cut_5.loc[i,'Qi_3']
    cut_5.loc[i+1,'Qi_5'] = cut_5.loc[i,'Qi_4']
    cut_5.loc[i+1,'Qi_6'] = cut_5.loc[i,'Qi_5']
    cut_5.loc[i+1,'Qi_7'] = cut_5.loc[i,'Qi_6']
    cut_5.loc[i+1,'Qi_8'] = cut_5.loc[i,'Qi_7']

### 结果提交保存

In [20]:
yhat = np.vstack((np.array(res).reshape(-1,),np.array(res2).reshape(-1,),np.array(res3).reshape(-1,),np.array(res4).reshape(-1,),np.array(res5).reshape(-1,)))
submit = pd.read_csv('./submission_sample.csv',index_col=0)
for i in range(len(yhat)):
    submit.iloc[i] = yhat[i]
submit.to_csv("./submit_result_notebook.csv")

In [21]:
# 打印结果
print(yhat)

[[0.07231856 0.07783443 0.08994661 0.10403395 0.08797615 0.0774916
  0.0958493  0.09338269 0.0522257  0.05085063 0.05590374 0.0741131
  0.08083397 0.08234105 0.09585103 0.09108399 0.04639896 0.04262129
  0.04326766 0.05756588 0.07294454 0.07708597 0.08974921 0.08768414
  0.04704357 0.04014701 0.04084164 0.05468963 0.07095588 0.073655
  0.0852921  0.08771224 0.04738443 0.03951432 0.04045887 0.05444195
  0.07031189 0.07217659 0.08385875 0.08608842 0.04705105 0.04005095
  0.04044207 0.05424767 0.07025752 0.07217859 0.08233523 0.08590812
  0.04704898 0.04004665 0.04050621 0.05445835 0.07026524 0.07215828
  0.08202857 0.08552902]
 [0.17026502 0.16063447 0.15335194 0.17831549 0.15909378 0.15289691
  0.18008282 0.19300454 0.11336044 0.10422385 0.10578583 0.14957715
  0.15566803 0.14467102 0.17368673 0.18719243 0.09413693 0.07635691
  0.07904201 0.11605363 0.13907717 0.13072418 0.16765797 0.18162082
  0.08095356 0.06687074 0.06444722 0.09812918 0.12478219 0.124923
  0.14665493 0.16619592 0.072